# 🔐 SecureLink: One-Time Encrypted Link Generator (Colab Demo)
Simulate the AWS Lambda-based secure link generator, which supports self-destructing messages.

In [ ]:

import json
import uuid
import time

# Mock database (acts like DynamoDB)
mock_db = {}

# Function to create a secure link
def create_secure_link(message, type_="text", expires_in=10):
    link_id = str(uuid.uuid4())
    ttl = int(time.time()) + (expires_in * 60)
    
    mock_db[link_id] = {
        'message': message,
        'type': type_,
        'expiresAt': ttl
    }
    
    return {
        "statusCode": 200,
        "body": json.dumps({
            "url": f"https://mock-api.com/secure-link?id={link_id}"
        })
    }

# Function to read and auto-delete secure link
def read_secure_link(link_id):
    if link_id not in mock_db or mock_db[link_id]['expiresAt'] < time.time():
        return {
            "statusCode": 404,
            "body": json.dumps({"error": "Link expired or not found"})
        }
    
    data = mock_db[link_id]
    del mock_db[link_id]  # Self-destruct after viewing

    return {
        "statusCode": 200,
        "body": json.dumps({
            "message": data['message'],
            "type": data['type']
        })
    }


In [ ]:

# ✅ Step 1: Create a Secure Link
create_response = create_secure_link("This is a secret message", expires_in=1)
create_response_json = json.loads(create_response["body"])
secure_link_id = create_response_json["url"].split('=')[-1]
print("🔗 Secure Link ID:", secure_link_id)
print("🔗 URL:", create_response_json["url"])


In [ ]:

# ✅ Step 2: Read the Secure Link (First Time)
read_response = read_secure_link(secure_link_id)
print("📩 Message Output:", json.loads(read_response["body"]))


In [ ]:

# ⚠️ Step 3: Try Reading Again (Should Show Expired)
read_again_response = read_secure_link(secure_link_id)
print("⛔ Second Read:", json.loads(read_again_response["body"]))
